In [1]:
import os
from datetime import datetime
from ltlcross_runner import LtlcrossRunner
import spot
spot.setup()

# Comparison of LTL to deterministic automata translators 

$\newcommand{\F}{\mathsf{F}}\newcommand{\G}{\mathsf{G}}\newcommand{\U}{\mathsf{U}}\newcommand{\X}{\mathsf{X}}$
## Tools
We have 
* tools that translate \[fragments\] of LTL into (generalized) Rabin automata:
 - [Rabinizer 3.1](https://www7.in.tum.de/~kretinsk/rabinizer3.html)
 - [Rabinizer 4](https://www7.in.tum.de/~kretinsk/rabinizer4.html)
 - [LTL3DRA](https://sourceforge.net/projects/ltl3dra/) v.0.2.5 \[LTL$\smallsetminus\G$($\U\X$)\]
* tools that chain translation of LTL into cut-deterministic Büchi automata (aka LDBA) or into deterministic Rabin automata with a construction that create deterministic parity automaton. They avoid Safra-like determinization. Both are from the [owl/Rabinizer4 library](https://www7.in.tum.de/~kretinsk/rabinizer4.html)
 - ltl2dpa --mode=ldba
 - ltl2dpa --mode=rabinizer (relies on transition-based index appearance record)
* tools that perform Safra-based determinization of N\[G\]BA into {Rabin/Parity} automata that use intermediete NBA
 - [ltl2dstar](http://ltl2dstar.de) \[NBA only\] {Rabin)}
 - autfilt from [Spot](https://spot.lrde.epita.fr/) 2.4.3 {parity}
 - autfilt from [Spot](https://spot.lrde.epita.fr/) 2.4.3.dev (to be 2.5 in the future) {parity}
 - here we use the following LTL2TGBA/LTL2NBA translators:
   - ltl2tgba from [Spot](https://spot.lrde.epita.fr/) 2.4.3
   - [LTL3BA](https://sourceforge.net/p/ltl3ba/) v. 1.1.3; we use LTL3BA in two settings:
     - `ltl3ba` {LTL3BA}
     - `ltl3ba -d` {LTL3BAd} which prefer more deterministic automata to smaller ones
 - we further use experimentaly for autfilt the LTL to Emerson-Lei automata translator
   - [LTL3TELA](https://github.com/jurajmajor/ltl3tela) v.1.1.0
* Safra's based determinization into deterministic parity automata that uses information about the LTL formula
 - `ltl2tgba -DG` from [Spot](https://spot.lrde.epita.fr/) 2.4.4
 - `ltl2tgba -DG`from [Spot](https://spot.lrde.epita.fr/) 2.4.4.dev (to be 2.5 in the future)

In [2]:
mkdir data_check formulae -p

In [3]:
if not os.path.exists('formulae/random_full.ltl'):
    %run Formulae.ipynb

In [4]:
%run Tools.ipynb

In [5]:
runners = {}
cols=["states","transitions","acc","time","nondet_states"]
for source in ('literature','random'):
    for t in ('full','ltl3dra'):
        name = '{}_{}'.format(source,t)
        tools = full_tools if t=='full' else ltl3dra_tools
        runners[name] = \
            LtlcrossRunner(tools,\
                    res_filename='data_check/{}.csv'.format(name),\
                    formula_files=['formulae/{}.ltl'.format(name)],\
                    cols=cols)

name = 'random_fg'
tools = ltl3dra_tools
runners[name] = LtlcrossRunner(tools,\
                    res_filename='data_check/{}.csv'.format(name),\
                    formula_files=['formulae/{}.ltl'.format(name)],\
                    cols=cols)

In [6]:
rerun = False

In [7]:
for name in runners.keys():
    #if 'random' in name:
    #    continue
    print('{}: Working on {}'.format(datetime.now().strftime('[%d.%m.%Y %T]'),name))
    if rerun:
        runners[name].run_ltlcross(timeout='120',
                check=True,
                lcr='/home/xblahoud/spot_dev/bin/ltlcross')
    runners[name].parse_results()

[08.01.2018 09:17:04]: Working on literature_full
[08.01.2018 09:17:05]: Working on literature_ltl3dra
[08.01.2018 09:17:15]: Working on random_full


FileNotFoundError: data_check/random_full.csv

There was an core dump during the ltlcross run on the formula number 71. I have removed the problematic formula number 71 and put True instead into the file `formulae/random_full_check.ltl`. The core dump was reported to ADL. This should be solved in the new

And then it happend again on formula 309. So that one is substituted by `False` in the file `formulae/random_full_check_#3.ltl`.

In [11]:
for name in runners.keys():
    #if 'random' in name:
    #    continue
    print('{}: Working on {}'.format(datetime.now().strftime('[%d.%m.%Y %T]'),name))
    if rerun:
        runners[name].run_ltlcross(timeout='120',check=True)
    runners[name].parse_results()

[08.01.2018 15:07:39]: Working on literature_full
[08.01.2018 15:07:39]: Working on literature_ltl3dra
[08.01.2018 15:07:43]: Working on random_full
[08.01.2018 15:07:44]: Working on random_fg
[08.01.2018 15:07:45]: Working on random_ltl3dra
